In [ ]:
# sudo chmod 777 /dev/ttyACM0
# sudo chmod 777 /dev/ttyACM1

In [1]:
%matplotlib notebook

# run this in a Jupyter (IPython) Notebook!
# modified from http://www.lebsanft.org/?p=48
# http://pyserial.readthedocs.org/en/latest/pyserial_api.html
import serial
import numpy as np
from matplotlib import pyplot as plt
from time import time

# If you're not using Linux, you'll need to change this
# check the Arduino IDE to see what serial port it's attached to
ser = serial.Serial('/dev/ttyUSB0', 115200)

'''
# set plot to animated
plt.ion()
'''

start_time = time()
timepoints = []
ydata = []
yydata = []
yrange = [-0.1,5.1]
view_time = 4 #seconds of data to view at once
duration = 700 #total seconds to collect data

'''
fig1 = plt.figure()
# http://matplotlib.org/users/text_props.html
fig1.suptitle('live updated data', fontsize='18', fontweight='bold')
plt.xlabel('time, seconds', fontsize='14', fontstyle='italic')
plt.ylabel('potential, volts', fontsize='14', fontstyle='italic')
plt.axes().grid(True)
line1, = plt.plot(ydata,marker='o',markersize=4,linestyle='none',markerfacecolor='red')
plt.ylim(yrange)
plt.xlim([0,view_time])
'''

# flush any junk left in the serial buffer
ser.flushInput()
# ser.reset_input_buffer() # for pyserial 3.0+
run = True

# collect the data and plot a moving frame
while run:
    ser.reset_input_buffer()
    data = ser.readline().decode("utf-8").split(' ')
    
    # sometimes the incoming data is garbage, so just 'try' to do this
    try:
        # store the entire dataset for later
        ydata.append(float(data[0]))
        yydata.append(float(data[1]))
        timepoints.append(time()-start_time)
        current_time = timepoints[-1]
        
        '''
        # update the plotted data
        line1.set_xdata(timepoints)
        line1.set_ydata(ydata)
        '''
        
        '''
        # slide the viewing frame along
        if current_time > view_time:
            plt.xlim([current_time-view_time,current_time])
        '''
            
        # when time's up, kill the collect+plot loop
        if timepoints[-1] > duration:
            run=False
    
    # if the try statement throws an error, just do nothing
    except: pass
    
'''    
    # update the plot
    fig1.canvas.draw()

# plot all of the data you collected
fig2 = plt.figure()
# http://matplotlib.org/users/text_props.html
fig2.suptitle('complete data trace', fontsize='18', fontweight='bold')
plt.xlabel('time, seconds', fontsize='14', fontstyle='italic')
plt.ylabel('potential, volts', fontsize='14', fontstyle='italic')
plt.axes().grid(True)

plt.plot(timepoints, ydata,marker='o',markersize=4,linestyle='none',markerfacecolor='red')
plt.ylim(yrange)
fig2.show()
'''

ser.close()

In [ ]:
import serial
import numpy as np
from matplotlib import pyplot as plt
from time import time

In [ ]:
ser = serial.Serial('/dev/ttyUSB0', 115200)

In [ ]:
# flush any junk left in the serial buffer
ser.flushInput()

In [ ]:
ser.reset_input_buffer()
data = ser.readline().decode("utf-8").split(' ')
data

In [ ]:
ser.readline().decode("utf-8").split(' ')

In [ ]:
len(yydata)

In [2]:
ydata_par = ydata #ydata[1:][::2] # slicing function
ydata_impar = yydata #ydata[::2] # slicing function

In [3]:
Q = zip(ydata_par, ydata_impar)

In [4]:
import xlsxwriter

workbook = xlsxwriter.Workbook('motor.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write('A1', 'time')
worksheet.write('B1', 'output')
worksheet.write('C1', 'input')

k  = 1
Ts = 0.2

for output, entry in zip(ydata_par, ydata_impar):
    worksheet.write(k, 0, Ts * k)
    worksheet.write(k, 1, output)
    worksheet.write(k, 2, entry)
    k += 1
    
workbook.close()